In [11]:
import cv2
import numpy as np
import os

# Funkcja Non-Maximum Suppression
def non_max_suppression(boxes, scores, threshold):
    selected_boxes = []
    selected_scores = []
    selected_indices = []
    areas = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
    sorted_indices = np.argsort(scores)[::-1]

    while len(sorted_indices) > 0:
        selected_index = sorted_indices[0]
        selected_boxes.append(boxes[selected_index])
        selected_scores.append(scores[selected_index])
        selected_indices.append(selected_index)

        intersection_x1 = np.maximum(boxes[selected_index, 0], boxes[sorted_indices[1:], 0])
        intersection_y1 = np.maximum(boxes[selected_index, 1], boxes[sorted_indices[1:], 1])
        intersection_x2 = np.minimum(boxes[selected_index, 2], boxes[sorted_indices[1:], 2])
        intersection_y2 = np.minimum(boxes[selected_index, 3], boxes[sorted_indices[1:], 3])

        intersection_width = np.maximum(0, intersection_x2 - intersection_x1)
        intersection_height = np.maximum(0, intersection_y2 - intersection_y1)

        intersection_area = intersection_width * intersection_height
        union_area = areas[selected_index] + areas[sorted_indices[1:]] - intersection_area
        iou = intersection_area / union_area

        filtered_indices = np.where(iou <= threshold)[0]
        sorted_indices = sorted_indices[filtered_indices + 1]

    return np.array(selected_boxes), np.array(selected_scores), np.array(selected_indices)



# Ścieżki do plików YOLO
weights_path = os.path.join("darknet", "cfg", "yolov3.weights")
cfg_path = os.path.join("darknet", "cfg", "yolov3.cfg")

# Wczytanie modelu YOLO
net = cv2.dnn.readNetFromDarknet(cfg_path, weights_path)

# Wczytanie nazw klas
with open("darknet/cfg/coco.names", "r") as f:
    classes = f.read().strip().split("\n")

# Wczytanie obrazu
image = cv2.imread("zdjecia/1.jpg")  
height, width = image.shape[:2]  

# Normalizacja obrazu i przekazanie przez sieć YOLO
blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)
output_layers_names = net.getUnconnectedOutLayersNames()
layer_outputs = net.forward(output_layers_names)

# Listy na bounding boxy i pewności
boxes = []
confidences = []
class_ids = []


# Analiza prognoz i rysowanie bounding boxów
for output in layer_outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.10:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, x + w, y + h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Konwersja danych na tablice numpy
boxes = np.array(boxes)
confidences = np.array(confidences)
class_ids = np.array(class_ids)

# Wywołanie funkcji NMS
selected_boxes = non_max_suppression(boxes, confidences, threshold=0.5)


# Wywołanie funkcji NMS
selected_boxes, selected_scores, selected_indices = non_max_suppression(boxes, confidences, threshold=0.5)

# Rysowanie wybranych bounding boxów i umieszczanie tekstu na obrazie
for i, box in enumerate(selected_boxes):
    x, y, x_max, y_max = box
    cv2.rectangle(image, (x, y), (x_max, y_max), (0, 255, 0), 2)
    label = f"{classes[class_ids[selected_indices[i]]]}: {selected_scores[i]:.2f}"
    cv2.putText(image, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Wyświetlenie obrazu z wykrytymi obiektami
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()